In [1]:
import os
import requests
import copy
from bs4 import BeautifulSoup
from langchain_core.documents import Document
from dotenv import load_dotenv
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
import requests
from langchain_ai21 import AI21SemanticTextSplitter
from langchain_pinecone import PineconeVectorStore

C:\Users\elija\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
load_dotenv()

True

In [3]:
# Access the environment variables
openai_api_key = os.getenv('OPENAI_API_KEY')
ai_twentyone_api_key = os.getenv('AI_TWENTYONE_API_KEY')
pinecone_api_key = os.getenv('PINECONE_API_KEY')
pc_index_wit = os.getenv('PINECONE_WIT_SEMANTIC')

In [4]:
os.environ["AI21_API_KEY"] = ai_twentyone_api_key
os.environ['OPENAI_API_KEY'] = openai_api_key
os.environ['PINECONE_API_KEY'] = pinecone_api_key

In [34]:
docs = [
    Document(page_content="Food places in Mission Hill around Wentworth include: Mikes Donuts, Crispy Dough Pizza, Chilacates, Ginger Exchange, Mama's Place, Milkweed, Ak's Takout & Delivery, Brigham Circle Chinese Food, Flames Restaurant, Halal Indian Cuisine, Chacho's Pizza & Subs, IL Mondo Pizzeria, Josep's Sub Shop and Pizza, Laughing Monk Cafe, Montecristo Mexican Grill, Nachlo Mexican & Pakistani Cuisine, Lilly's Gourment Pasta, Papa's Pizza Company, Penguin Pizza, The Puddingstone Tavern, Solid Ground Cafe, TBaar, Tremont House of Pizza, University House of Pizza, Wok n Talk Pizza, Tavern of Tales, The Mission Bar & Grill, and TGI Fridays", metadata={"source": "https://explore.missionhillmainstreet.com/collections/restaurants-dining"}),
    Document(page_content="Mikes Donuts, a donuts, muffins, pastries, sandwiches, salads & soups served from a casual spot with sidewalk seats, especially the legendary honey-dipped variety, best enjoyed with hot java in a 20-ounce Mike’s mug you can take home to be your new weekend-coffee companion. On workdays, meanwhile, you can also grab a commute-friendly doughnut from Mike’s satellite kiosk inside the Forest Hills T station, a few stops away. Its located at 1524 Tremont St A, CROSSING, MA 02120, USA. Their number is +1 617-427-6828 and their website is http://www.mikesdonutsboston.com/", metadata={"source": "https://explore.missionhillmainstreet.com/collections/restaurants-dining"}),
    Document(page_content="Crispy Dough Pizzeria is a cozy, casual pizzeria serving a menu of pies, pastas & sandwiches, plus eclectic entrees located in 1514 Tremont St, Boston, MA 02120, USA. Their number is +1 617-445-7799 and their website is http://www.crispydoughpizzeria.com/", metadata={"source": "https://explore.missionhillmainstreet.com/collections/restaurants-dining"}),
    Document(page_content="We are CHILACATES, hardworking food lovers with a passion for the culinary Mexican experience. We focus on the taste sensations of every meal. We will strive to fill your lives with the most authentic Mexican street food experience. Serving you the freshest most local and natural ingredients that are available to us. “Because at the end of the day we just want you to be happy!”. We are located at 1482 A Tremont St, Boston, MA 02120, USA. Our number is +1 617-286-3888 and our website is http://www.chilacates.mx/", metadata={"source": "https://explore.missionhillmainstreet.com/collections/restaurants-dining"}),
    Document(page_content="Ginger Exchange has Eclectic menu of Asian dishes including sushi, stir-fry & vegetarian options in a modern, cozy spot located at 1625 Tremont St, Crossing, MA 02120, USA with their phone number of +1 617-739-8888 and their website https://www.gingerexchange.com/", metadata={"source": "https://explore.missionhillmainstreet.com/collections/restaurants-dining"}),
    Document(page_content="Mamas Place, a laid-back local outfit for gyros, spinach pie & Greek classics plus burgers located at 764 Huntington Ave #6321, Boston, MA 02115, USA. Their phone number is +1 617-566-1300 and their website is https://www.gomamasplace.com/", metadata={"source": "https://explore.missionhillmainstreet.com/collections/restaurants-dining"}),
    Document(page_content="Milkweed, a friendly and homey-feeling cafe with globe-trotting menu & breakfast served into early afternoon located at 1508 Tremont St, Boston, MA 02120, USA with their number being +1 617-516-8913 and their website being http://www.eatatmilkweed.com/", metadata={"source": "https://explore.missionhillmainstreet.com/collections/restaurants-dining"}),
    Document(page_content="Ak’s Takeout and Delivery’s Food quality and customer satisfaction are inarguably the two core policies of AKS Takeout. In today’s world of intense competition, our Key to success lies in delivering high quality service & fast food that in turn had earned us satisfied, repeating customer since years. The combined effects of Good food, Quality Service, tasty food has been influencing customers to give us five star ratings in Boston Area. Its located in 1427 Tremont St, Boston, MA 02120, USA. Their number is +1 617-541-0300 and their website is https://www.akstakeout.com/", metadata={"source": "https://explore.missionhillmainstreet.com/collections/restaurants-dining"}),
    Document(page_content="Brigham Circle Chinese Food is a mix of standard Chinese dishes & weekday lunch deals on offer in this low-key neighborhood eatery located at 728 Huntington Ave, Boston, MA 02115, USA. Their number is +1 617-278-2000 and their website is http://orderbrighamcirclechinesefood.com/", metadata={"source": "https://explore.missionhillmainstreet.com/collections/restaurants-dining"}),
    Document(page_content="We are Flames Restaurant. Each of our four locations has its own unique style while providing the same welcoming Caribbean experience through the authentic cuisine, colorful decor coupled with vibrant artwork and pictures of popular West Indian entertainers, some of whom have enjoyed our meals.. Extensive catering service is available and orders can be customized to suit any social or corporate event. Contact us at (617) 734-1911. Take some time to tour our site and please stop in for a bite at your convenience. We would also love to discuss ways to make your next event a memorable taste experience. We are located in 746 Huntington Ave, Boston, MA 02115, USA. Our number is +1 617-734-1911", metadata={"source": "https://explore.missionhillmainstreet.com/collections/restaurants-dining"}),
    Document(page_content="Halal Indian Cuisine introduces homestyle Indian cooking, including seafood, tandoori & vegetarian options, in a modest storefront located in 736 Huntington Ave, Boston, MA 02115, USA. Their number is +1 617-232-5000 and their website is http://www.halalindiancuisineboston.com/", metadata={"source": "https://explore.missionhillmainstreet.com/collections/restaurants-dining"}),
    Document(page_content="Chachos Pizza & Subs is a long-running corner joint serving Argentinean pizza, sandwiches & calzones, plus dinner entrees located in Tremont St, Roxbury Crossing, MA 02120, USA. Their number is +1 617-445-6738 and their website is https://www.chachospizzasubs.com/", metadata={"source": "https://explore.missionhillmainstreet.com/collections/restaurants-dining"}),
    Document(page_content="We are IL Mondo Pizzeria. We take great pride in providing the highest quality food and service for a quality dining experience. We use only the freshest ingredients and the finest dough to create our pizzas. If you have any food allergies or require specific delivery needs, let us know. We are happy to accommodate you. We are located at 738 Huntington Ave, Boston, MA 02115, USA. Our number is +1 617-277-7161 and our website is http://www.ilmondopizza.com/ . Thank you for choosing Il Mondo Pizzeria.", metadata={"source": "https://explore.missionhillmainstreet.com/collections/restaurants-dining"}),
    Document(page_content="We are Joseph’s Sub Shop and Pizza. This unpretentious pizza parlor offers classic pies & slices, plus burgers, sandwiches & salads located at 8 S Huntington Ave, Jamaica Plain, MA 02130, USA. Our number is +1 617-731-9880 ", metadata={"source": "https://explore.missionhillmainstreet.com/collections/restaurants-dining"}),
    Document(page_content="Laughing Monk Café serves intricate sushi rolls & inventive Thai cuisine offered in a fashionable backdrop with a modern vibe at 737 Huntington Ave, Boston, MA 02115, USA. Our number is +1 617-232-8000 and our website is http://www.laughingmonkcafe.com/", metadata={"source": "https://explore.missionhillmainstreet.com/collections/restaurants-dining"}),
    Document(page_content="Montecristo Mexican grill is a mix of regional Mexican dishes served in a basic setting, with an extensive delivery area located in 748 Huntington Ave #A, Boston, MA 02115, USA. Our number is +1 617-275-4989 and our website is https://www.montecristomissionhill.com/", metadata={"source": "https://explore.missionhillmainstreet.com/collections/restaurants-dining"}),
    Document(page_content="Nachlo Indian cuisine Roxbury is where you can relish popular Indian Pakistani delicacies such as the Tandoori Chicken, Biryani, Mutton Rogan Josh, Paneer Tikka, Kebabs, Curries, Naan, Dosa, and much more. Nachlo Mexican Roxbury is known for really good food and good prices serving delicious Tacos, Hamburgers, Enchiladas, Burrito and a vast variety of kid’s menu full of nutrition attracting people of all age groups. Our menu is set to bring in mouth -watering dishes inspired by the street food culture of Mexico – in its signature style. Our Curries are full of variety like watery, dry, red, green, hot, or really hot—explore our menu to choose from. We are located in 1443 Tremont St, Roxbury, MA 02120, USA. Our number is +1 617-397-3200 and our website is https://www.nachlocuisineroxbury.com/", metadata={"source": "https://explore.missionhillmainstreet.com/collections/restaurants-dining"}),
    Document(page_content="Lilly’s Gourmet Pasta is a Snug corner storefront serving handmade pastas & raviolis (with a variety of fillings) to go. We’re located at 1528 Tremont St, Boston, MA 02120, USA. Our number is +1 617-427-8080 and our website is http://www.lillysgourmetpasta.com/", metadata={"source": "https://explore.missionhillmainstreet.com/collections/restaurants-dining"}),
    Document(page_content="Papa’s Pizza Company is Serving Thin Crust Pizzas with Homemade pizza dough and sauces. We offer delivery, take out, and late night service to Mission Hill and the surrounding areas! Our goal is to offer our customers a diverse menu and enjoyable experience by using made from scratch ingredients like our signature sauce and pizza dough prepared daily in-house. We are located in 682 Huntington Ave, Boston, MA 02115, USA. Our number is +1 617-648-7272 and our website is http://www.papaspizzacompany.com/ . We look forward to cooking for you!", metadata={"source": "https://explore.missionhillmainstreet.com/collections/restaurants-dining"}),
    Document(page_content="Penguin Pizza is nothing fancy, just pizza, beer, and a whole lot of a good stuff. Informal spot for thin-crust pies with inventive toppings & a huge selection of bottled beers. We are located at 735 Huntington Ave, Boston, MA 02115, USA. Our number is +1 617-277-9200 and our website is http://www.thepenguinpizza.com/", metadata={"source": "https://explore.missionhillmainstreet.com/collections/restaurants-dining"}),
    Document(page_content="We are the Puddingstone Tavern. Come enjoy a cold beer or cocktail on our outdoor patio! Check out our Large Selection of Beers on Tap at 1592 Tremont St, Boston, MA 02120, USA. Our number is +1 617-608-3152 and our website is http://www.thepuddingstonetavern.com/", metadata={"source": "https://explore.missionhillmainstreet.com/collections/restaurants-dining"}),
    Document(page_content="Local Mini Mart located at 1576 Tremont St, Boston, MA 02120, USA", metadata={"source": "https://explore.missionhillmainstreet.com/collections/restaurants-dining"}),
    Document(page_content="Locally owned Solid Ground Cafe offers respite and refueling to Mission Hill's bustling community. Coffee, espresso, all day breakfast, lunch. Currently offering takeout only. Masks must be worn. No consumption indoors. Located at 742 Huntington Ave, Boston, MA 02115, USA. Our number is +1 617-487-5557 and our website is http://solidgroundcafe.com/ .", metadata={"source": "https://explore.missionhillmainstreet.com/collections/restaurants-dining"}),
    Document(page_content="We are TBaar. Our ingredients are shipped directly from a company located in Taiwan. We only choose the finest premium tea such as Earl Grey, Jasmine Green tea, and roasted oolong tea. We have distribution contracts with high quality tea growers around the world. We are located in 754 Huntington Ave, Boston, MA 02115, USA. Our number is +1 617-487-5205.", metadata={"source": "https://explore.missionhillmainstreet.com/collections/restaurants-dining"}),
    Document(page_content="Tremont House of Pizza, a Cozy, long-running choice with booths & patio seating for classic pizzas, pastas & subs located in 1590 Tremont St, Boston, MA 02120, USA. Our number is +1 617-566-6120 and our website is http://www.missionhillthop.com/", metadata={"source": "https://explore.missionhillmainstreet.com/collections/restaurants-dining"}),
    Document(page_content="At University House of Pizza, we pride ourselves on preparing a delicious meal for every customer, every time. Our ingredients are fresh, and our staff is friendly and courteous. For your convenience, we offer dine-in, pickup and delivery service. Check out our menu and place your order now. Let us show you that our customers' satisfaction is our number one priority! We are located at 452 Huntington Ave, Boston, MA 02115, USA . Our number is +1 617-442-9280", metadata={"source": "https://explore.missionhillmainstreet.com/collections/restaurants-dining"}),
    Document(page_content="We are Wok n Talk Pizza, an Asian open kitchen where everything is fresh and made to order right before your eyes. At Wok n’ Talk, we serve delicious, healthy food. We use no MSG and cook with 100% vegetable oil. Wok ‘n Talk food is available for eat-in, takeout or delivery. We have complimentary WiFi and parking. We also offer catering services, so please let us know about your next event. We are also located at 23 S Huntington Ave, Boston, MA 02130, USA. Our number is +1 617-487-8262 and our website is http://www.wokntalkboston.com/", metadata={"source": "https://explore.missionhillmainstreet.com/collections/restaurants-dining"}),
    Document(page_content="Tavern of Tales is an interactive bar and board game experience, located at 1478 Tremont St., Boston, between the Roxbury Crossing and Brigham Circle T stops. Tavern currently offering seven different immersive, sound-enhanced game-play adventures for players at all levels! We are located at 1478 Tremont St, Boston, MA 02120, USA. Our number is +1 617-708-0294 and our website is https://www.tavernoftales.com/", metadata={"source": "https://explore.missionhillmainstreet.com/collections/restaurants-dining"}),
    Document(page_content="The Mission Bar & Grill is a neighborhood pub offering beer & bar bites like steak tips & fish 'n' chips in a basic brick space. We are located at 724 Huntington Ave, Boston, MA 02115, USA. Our number is +1 617-566-1244 and our website is http://www.themissionbar.com/", metadata={"source": "https://explore.missionhillmainstreet.com/collections/restaurants-dining"}),
    Document(page_content="TGI Fridays, a restaurant in Mission Hill and near Wentworth, reputation as the best corner bar in town started in Manhattan in 1965—and it continues today right here in Massachusetts where we’ve been serving up good times in Boston. Located near Tremont St & Calumet St, we’re proudly bringing you the best local mix of food, fun, and handcrafted cocktails. A tasty dinner is just minutes away from Museum of Fine Arts and Fenway Park. Whether you bring friends or come to make friends, stop by and see us at TGI Fridays. We are located at 1626 Tremont St, Boston, MA 02120, USA. Our number is +1 617-734-1047 and our website is https://locations.tgifridays.com/ma/boston/1626-tremont-st.html?utm_source=google&utm_medium=local&utm_campaign=google-local", metadata={"source": "https://explore.missionhillmainstreet.com/collections/restaurants-dining"}),
]

In [35]:
semantic_splitter = AI21SemanticTextSplitter(
    chunk_size=50
)
all_splits = semantic_splitter.split_documents(docs)

In [36]:
all_splits

[Document(page_content="Food places in Mission Hill around Wentworth include: Mikes Donuts, Crispy Dough Pizza, Chilacates, Ginger Exchange, Mama's Place, Milkweed, Ak's Takout & Delivery, Brigham Circle Chinese Food, Flames Restaurant, Halal Indian Cuisine, Chacho's Pizza & Subs, IL Mondo Pizzeria, Josep's Sub Shop and Pizza, Laughing Monk Cafe, Montecristo Mexican Grill, Nachlo Mexican & Pakistani Cuisine, Lilly's Gourment Pasta, Papa's Pizza Company, Penguin Pizza, The Puddingstone Tavern, Solid Ground Cafe, TBaar, Tremont House of Pizza, University House of Pizza, Wok n Talk Pizza, Tavern of Tales, The Mission Bar & Grill, and TGI Fridays", metadata={'source': 'https://explore.missionhillmainstreet.com/collections/restaurants-dining', 'source_type': 'normal_text'}),
 Document(page_content='Mikes Donuts, a donuts, muffins, pastries, sandwiches, salads & soups served from a casual spot with sidewalk seats, especially the legendary honey-dipped variety, best enjoyed with hot java in a

In [32]:
embeddings = OpenAIEmbeddings()

In [33]:
docsearch = PineconeVectorStore.from_documents(all_splits, embeddings, index_name=pc_index_wit)